# Medical AI Analysis System

AWS Bedrock + LangGraph + OpenSearch 기반 의료 이미지 분석 시스템

## LangGraph 멀티에이전트 워크플로우
1. **Disease Detection Agent**: 멀티모달 이미지 분석
2. **Similar Search Agent**: OpenSearch 유사사례 검색
3. **Report Generation Agent**: 의료 보고서 생성
4. **Alert System Agent**: 응급도 판단 및 알림

## 1. 환경 설정 및 라이브러리 Load

In [13]:
# 필수 라이브러리 설치 (처음 실행시에만)
# !pip install -r requirements.txt

import os
import json
import base64
import uuid
from datetime import datetime
from typing import Dict, Any, List

# 시각화
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import display, HTML, Image
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed

# LangGraph
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

# 설정 및 에이전트
from config import Config
from agents.disease_detection import DiseaseDetectionAgent
from agents.report_generation import ReportGenerationAgent
from agents.alert_system import AlertSystemAgent
from agents.similar_search import SimilarSearchAgent

# OpenSearch 통합 (실제 클러스터 연결)
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))
from opensearch_multimodal import opensearch_multimodal

print("====================================================")
print(" 라이브러리 로드 완료 (OpenSearch 통합)")
print("====================================================")

----------------------------------------------------
 라이브러리 로드 완료 (OpenSearch 통합)
----------------------------------------------------


## 2. 설정 확인 및 검증

In [17]:
# 환경 변수 확인
print("====================================================")
print(" 시스템 설정 확인\n")
print(f"AWS Region: {Config.AWS_REGION}")
print(f"S3 Bucket: {Config.S3_BUCKET}")
print(f"Bedrock Model: {Config.BEDROCK_MODEL_ID}")
print(f"OpenSearch Endpoint: {os.getenv('OPENSEARCH_ENDPOINT', 'Default')}")
print(f"OpenSearch Index: {os.getenv('OPENSEARCH_INDEX', 'Default')}")
print(f"Email Alerts: {'활성화' if Config.ENABLE_EMAIL_ALERTS else '비활성화'}")

# 설정 검증
try:
    Config.validate()
    print("\n 설정 검증 완료")
except ValueError as e:
    print(f"\n 설정 오류: {e}")
    print("\n.env 파일을 확인하고 필수 값들을 설정해주세요:")
    print("- AWS_ACCESS_KEY_ID")
    print("- AWS_SECRET_ACCESS_KEY")
    print("- S3_BUCKET")

# OpenSearch 테스트
print("====================================================")
print(" OpenSearch 유사검색 테스트\n")
test_cases = opensearch_multimodal.search_similar_cases("infarct")
print(f"유사사례 {len(test_cases)}개 발견")
for i, case in enumerate(test_cases[:3]):
    print(f"  {i+1}. {case['u_id']}: {case['diagnosis']} (유사도: {case['similarity_score']})")

print("====================================================")

----------------------------------------------------
 시스템 설정 확인

AWS Region: us-west-2
S3 Bucket: {Config.S3_BUCKET}
Bedrock Model: anthropic.claude-3-5-sonnet-20241022-v2:0
OpenSearch Endpoint: https://apl11lr5oawl8mcvnqoh.us-east-1.aoss.amazonaws.com
OpenSearch Index: medical-multimodal-cases
Email Alerts: 비활성화

 설정 검증 완료
----------------------------------------------------
 OpenSearch 유사검색 테스트

유사사례 2개 발견
  1. MPX1205: Left PICA Infarct confirmed with MRI.  MR angiogram images and CT angiogram images demonstrated a patent vertebral system and patent left PICA.  A focal stenosis was detected at the origin of the left posterior inferior cerebellar artery. (유사도: 1.0)
  2. MPX1420: Ependymoma (유사도: 0.27564444)
----------------------------------------------------


## 3. LangGraph 워크플로우 정의 (OpenSearch 통합)

In [18]:
# 상태 정의 (OpenSearch 결과 추가)
class MedicalAnalysisState(TypedDict):
    case_id: str
    image_data: bytes
    keywords: str
    diagnosis_result: Dict[str, Any]
    similar_result: Dict[str, Any]  # OpenSearch 유사검색 결과
    report_result: Dict[str, Any]
    alert_result: Dict[str, Any]
    workflow_steps: List[str]
    final_result: Dict[str, Any]

# 에이전트 초기화 (OpenSearch 에이전트 추가)
disease_agent = DiseaseDetectionAgent()
similar_agent = SimilarSearchAgent()
report_agent = ReportGenerationAgent()
alert_agent = AlertSystemAgent()

# 에이전트 함수들
def disease_detection_node(state: MedicalAnalysisState) -> MedicalAnalysisState:
    """질병 진단 에이전트 실행"""
    print(" [Agent 1/4] Disease Detection Agent 실행 중...")
    
    result = disease_agent.analyze_image(state["image_data"], state["keywords"])
    
    state["diagnosis_result"] = result
    state["workflow_steps"].append(" 질병 진단 완료")
    
    if result["success"]:
        print(f"    진단: {result['diagnosis']} (신뢰도: {result['confidence']})")
    else:
        print(f"    진단 실패: {result.get('error', 'Unknown error')}")
    
    return state

def similar_search_node(state: MedicalAnalysisState) -> MedicalAnalysisState:
    """OpenSearch 유사사례 검색 에이전트 실행"""
    print(" [Agent 2/4] Similar Search Agent 실행 중...")
    
    result = similar_agent.search_similar_cases(
        state["diagnosis_result"], 
        state["keywords"], 
        state["image_data"]
    )
    
    state["similar_result"] = result
    state["workflow_steps"].append(" 유사사례 검색 완료")
    
    if result["success"]:
        print(f"    유사사례: {result['total_found']}개 발견 (평균 유사도: {result['average_similarity']})")
        for case in result['similar_cases'][:2]:
            print(f"      - {case['case_id']}: {case['diagnosis']} (유사도: {case['similarity_score']})")
    else:
        print(f"    유사검색 실패: {result.get('error', 'Unknown error')}")
    
    return state

def report_generation_node(state: MedicalAnalysisState) -> MedicalAnalysisState:
    """보고서 생성 에이전트 실행"""
    print(" [Agent 3/4] Report Generation Agent 실행 중...")
    
    result = report_agent.generate_report(state["diagnosis_result"])
    
    state["report_result"] = result
    state["workflow_steps"].append(" 보고서 생성 완료")
    
    if result["success"]:
        print(f"    보고서: {result['sentence_count']}문장 생성")
    else:
        print(f"    보고서 생성 실패: {result.get('error', 'Unknown error')}")
    
    return state

def alert_system_node(state: MedicalAnalysisState) -> MedicalAnalysisState:
    """알림 시스템 에이전트 실행"""
    print(" [Agent 4/4] Alert System Agent 실행 중...")
    
    result = alert_agent.evaluate_alert_need(state["diagnosis_result"])
    
    state["alert_result"] = result
    state["workflow_steps"].append(" 알림 판단 완료")
    
    if result["alert_needed"] == "yes":
        print(f"    * 응급 알림 필요: {result['alert_reason']}")
        if result.get("email_sent"):
            print(f"    📧 이메일 발송 완료: {result.get('email_message_id', '')}")
    else:
        print("    * 정상 범위: 응급 알림 불필요")
    
    return state

def compile_results_node(state: MedicalAnalysisState) -> MedicalAnalysisState:
    """최종 결과 컴파일 (OpenSearch 결과 포함)"""
    print("최종 결과 컴파일 중...")
    
    # 최종 결과 구성
    final_result = {
        "case_id": state["case_id"],
        "timestamp": datetime.now().isoformat(),
        
        # 5-1: 질병명
        "disease_name": state["diagnosis_result"].get("diagnosis", "진단 실패"),
        "confidence": state["diagnosis_result"].get("confidence", "0%"),
        
        # 5-2: OpenSearch 유사사례 검색 결과
        "similar_cases": state["similar_result"].get("similar_cases", []),
        "similar_total": state["similar_result"].get("total_found", 0),
        "similar_avg_score": state["similar_result"].get("average_similarity", 0),
        "most_common_diagnosis": state["similar_result"].get("most_common_diagnosis", "N/A"),
        
        # 5-3: 다음 조치
        "next_actions": state["report_result"].get("next_actions", "추가 분석 필요"),
        
        # 5-4: 5문장 보고서
        "report": state["report_result"].get("report", "보고서 생성 실패"),
        
        # 5-5: 경고 알림
        "alert_needed": state["alert_result"].get("alert_needed", "no"),
        "alert_reason": state["alert_result"].get("alert_reason", ""),
        
        # 워크플로우 정보
        "workflow_steps": state["workflow_steps"],
        "opensearch_enhanced": True,
        "processing_success": all([
            state["diagnosis_result"].get("success", False),
            state["similar_result"].get("success", False),
            state["report_result"].get("success", False)
        ])
    }
    
    state["final_result"] = final_result
    state["workflow_steps"].append("결과 컴파일 완료")
    
    print(" 모든 에이전트 실행 완료 (OpenSearch 통합)")
    return state

# LangGraph 워크플로우
workflow = StateGraph(MedicalAnalysisState)

# 노드 추가
workflow.add_node("disease_detection", disease_detection_node)
workflow.add_node("similar_search", similar_search_node)  # OpenSearch 노드 추가
workflow.add_node("report_generation", report_generation_node)
workflow.add_node("alert_system", alert_system_node)
workflow.add_node("compile_results", compile_results_node)

# 엣지 설정
workflow.set_entry_point("disease_detection")
workflow.add_edge("disease_detection", "similar_search")
workflow.add_edge("similar_search", "report_generation")
workflow.add_edge("report_generation", "alert_system")
workflow.add_edge("alert_system", "compile_results")
workflow.add_edge("compile_results", END)

# 워크플로우 컴파일
medical_workflow = workflow.compile()

print("* LangGraph 워크플로우 구성 완료 (OpenSearch 통합)")

* LangGraph 워크플로우 구성 완료 (OpenSearch 통합)


## 4. 이미지 업로드 인터페이스

In [26]:
# 파일 업로드 위젯
file_upload = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='의료 이미지 업로드'
)

# 키워드 입력 위젯
keywords_input = widgets.Text(
    value='',
    placeholder='증상 등 정보를 입력하세요',
    description='키워드:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# 분석 버튼
analyze_button = widgets.Button(
    description=' AI 분석 시작',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='40px')
)

# 결과 출력 영역
output_area = widgets.Output()

# 업로드된 이미지 표시
def on_file_upload(change):
    if file_upload.value:
        with output_area:
            output_area.clear_output()
            
            # 파일 정보 추출
            uploaded_file = file_upload.value[0] if isinstance(file_upload.value, tuple) else file_upload.value
            
            if hasattr(uploaded_file, 'name'):
                print(f" 업로드된 파일: {uploaded_file.name}")
                image_data = uploaded_file.content
            else:
                # 딕셔너리 형태인 경우
                filename = list(uploaded_file.keys())[0]
                print(f" 업로드된 파일: {filename}")
                image_data = uploaded_file[filename]['content']
            
            # 이미지 표시 (크기 제한)
            display(Image(data=image_data, width=300))

file_upload.observe(on_file_upload, names='value')

# UI 구성
ui = widgets.VBox([
    widgets.HTML("<h3> 의료 이미지 업로드 (OpenSearch 통합)</h3>"),
    file_upload,
    keywords_input,
    analyze_button,
    output_area
])

display(ui)

## 5. 분석 실행 함수 (OpenSearch 통합)

In [27]:
def run_medical_analysis(image_data: bytes, keywords: str) -> Dict[str, Any]:
    """의료 분석 워크플로우 실행 (OpenSearch 통합)"""
    
    # 케이스 ID 생성
    case_id = str(uuid.uuid4())[:8]
    
    # 초기 상태 설정 (OpenSearch 결과 필드 추가)
    initial_state = {
        "case_id": case_id,
        "image_data": image_data,
        "keywords": keywords,
        "diagnosis_result": {},
        "similar_result": {},  # OpenSearch 결과
        "report_result": {},
        "alert_result": {},
        "workflow_steps": [],
        "final_result": {}
    }
    
    print(f" 의료 분석 시작 - Case ID: {case_id}")
    print(f" 키워드: {keywords}")
    print(f" OpenSearch 유사검색 포함")
    print("=" * 50)
    
    # LangGraph 워크플로우 실행
    try:
        final_state = medical_workflow.invoke(initial_state)
        return final_state["final_result"]
    except Exception as e:
        print(f" 워크플로우 실행 실패: {e}")
        return {
            "case_id": case_id,
            "error": str(e),
            "processing_success": False
        }

def display_results(result: Dict[str, Any]):
    """분석 결과 시각화 (OpenSearch 결과 포함)"""
    
    if not result.get("processing_success", False):
        display(HTML(f"<div style='color: red; font-size: 16px;'> 분석 실패: {result.get('error', 'Unknown error')}</div>"))
        return
    
    # OpenSearch 유사사례 HTML 생성
    similar_cases_html = ""
    if result.get('similar_cases'):
        similar_cases_html = "<ul>"
        for case in result['similar_cases'][:3]:
            similar_cases_html += f"""
            <li><strong>{case['case_id']}</strong>: {case['diagnosis']} 
                (유사도: {case['similarity_score']}, 
                 환자: {case['patient_info']['age']}세 {case['patient_info']['sex']})
                <br><small>{case['description']}</small>
            </li>
            """
        similar_cases_html += "</ul>"
    else:
        similar_cases_html = "<p>유사사례를 찾을 수 없습니다.</p>"
    
    # HTML 결과 구성 (OpenSearch 결과 포함)
    html_result = f"""
    <div style='border: 2px solid #4CAF50; border-radius: 10px; padding: 20px; margin: 10px 0; background-color: #f9f9f9;'>
        <h2 style='color: #2E7D32; margin-top: 0;'> 의료 분석 결과 (OpenSearch 통합)</h2>
        
        <div style='margin: 15px 0;'>
            <h3 style='color: #1976D2;'> Case ID: {result['case_id']}</h3>
            <p><strong>분석 시간:</strong> {result['timestamp']}</p>
            <p><strong>OpenSearch 강화:</strong>  활성화</p>
        </div>
        
        <div style='background-color: white; padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <h3 style='color: #D32F2F;'> 5-1. 질병명</h3>
            <p style='font-size: 18px; font-weight: bold;'>{result['disease_name']}</p>
            <p><strong>신뢰도:</strong> {result['confidence']}</p>
        </div>
        
        <div style='background-color: white; padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <h3 style='color: #F57C00;'> 5-2. OpenSearch 유사사례 검색 결과</h3>
            <p><strong>발견된 유사사례:</strong> {result['similar_total']}개</p>
            <p><strong>평균 유사도:</strong> {result['similar_avg_score']}</p>
            <p><strong>가장 흔한 진단:</strong> {result['most_common_diagnosis']}</p>
            <div><strong>상위 유사사례:</strong></div>
            {similar_cases_html}
        </div>
        
        <div style='background-color: white; padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <h3 style='color: #388E3C;'> 5-3. 다음 조치 제안</h3>
            <p>{result['next_actions']}</p>
        </div>
        
        <div style='background-color: white; padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <h3 style='color: #7B1FA2;'> 5-4. 의료 보고서 (5문장)</h3>
            <p style='line-height: 1.6;'>{result['report']}</p>
        </div>
        
        <div style='background-color: white; padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <h3 style='color: #C62828;'> 5-5. 경고 알림</h3>
            <p><strong>알림 필요:</strong> <span style='font-size: 18px; font-weight: bold; color: {'red' if result['alert_needed'] == 'yes' else 'green'};'>{result['alert_needed']}</span></p>
            {f"<p><strong>알림 사유:</strong> {result['alert_reason']}</p>" if result['alert_needed'] == 'yes' else ""}
        </div>
        
        <div style='background-color: #E3F2FD; padding: 15px; border-radius: 5px; margin: 10px 0;'>
            <h3 style='color: #1565C0;'> 워크플로우 단계</h3>
            <ul>
                {''.join([f"<li>{step}</li>" for step in result['workflow_steps']])}
            </ul>
        </div>
    </div>
    """
    
    display(HTML(html_result))

# 분석 버튼 클릭 이벤트
def on_analyze_click(button):
    with output_area:
        if not file_upload.value:
            print("이미지를 먼저 업로드해주세요.")
            return
        
        if not keywords_input.value.strip():
            print("키워드를 입력해주세요.")
            return
        
        # 이미지 데이터 추출
        uploaded_file = file_upload.value[0] if isinstance(file_upload.value, tuple) else file_upload.value
        
        if hasattr(uploaded_file, 'content'):
            image_data = uploaded_file.content
        else:
            file_info = list(uploaded_file.values())[0]
            image_data = file_info['content']
            
        keywords = keywords_input.value.strip()
        
        # 분석 실행
        output_area.clear_output()
        result = run_medical_analysis(image_data, keywords)
        
        print("\n" + "=" * 50)
        display_results(result)

analyze_button.on_click(on_analyze_click)

print("=============================================")
print(" \n분석 시스템 준비 완료 (OpenSearch 통합)\n")
print("=============================================")
print(" \n위의 인터페이스에서 이미지를 업로드하고 키워드를 입력한 후 ' AI 분석 시작' 버튼을 클릭하세요.\n")
print("=============================================")
print(" \nOpenSearch 유사검색이 자동으로 실행되어 5-2 결과에 표시됩니다.\n")
print("=============================================")

 
분석 시스템 준비 완료 (OpenSearch 통합)

 
위의 인터페이스에서 이미지를 업로드하고 키워드를 입력한 후 ' AI 분석 시작' 버튼을 클릭하세요.

 
OpenSearch 유사검색이 자동으로 실행되어 5-2 결과에 표시됩니다.

